# TDA Analysis

**Goal:** Compute persistent homology for clean CIFAR-10 features and visualize topological structure

### Overview
In this notebook, we will:
1. Load the clean features (50-dim PCA-reduced)
2. Compute persistence diagrams for H0 (connected components) and H1 (loops)
3. Visualize persistence diagrams
4. Analyze Betti curves
5. Extract topological statistics
6. Save diagrams for later comparison with adversarial features

---

### Why TDA?
Traditional ML looks at distances and gradients in feature space. TDA reveals the shape of the data:
- **H0 (homology dim 0):** Connected components: how clustered is the data?
- **H1 (homology dim 1):** Loops/cycles: are there circular structures?

**Our hypothesis:** Adversarial perturbations will fragment clusters (more H0 features) and **destroy** loops (fewer H1 features).

### Setup and Imports

In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.models.feature_extractor import FeatureExtractor
from src.tda.persistence import compute_persistence, save_diagrams, get_persistence_stats
from src.utils.plotting import plot_persistence_diagram, plot_betti_curve

np.random.seed(111)

print("Successfully imported everything")

Successfully imported everything


### Load Clean Features

We'll use the 50-dimensional PCA-reduced features from `02_feature_extraction.ipynb`. These features were extracted from 45,000 clean CIFAR-10 training images using ResNet50.

In [2]:
data = np.load('../results/features/clean_features_reduced.npz')
clean_features = data['features']
clean_labels = data['labels']

print(f"Clean features shape: {clean_features.shape}")
print(f"Feature range: [{clean_features.min():.4f}, {clean_features.max():.4f}]")
print(f"Labels: {clean_labels.shape}")

Clean features shape: (45000, 50)
Feature range: [-2.4786, 1.6122]
Labels: (45000,)


### Subsample for TDA

Computing persistent homology on 45,000 points is computationally expensive. Therefore, let's use a subset for initial analysis:
- **10,000 samples** for quick experimentation
- Later we can scale up to full dataset if needed

In [3]:
n_samples = 10000
indices = np.random.choice(len(clean_features), n_samples, replace=False)
clean_features_subset = clean_features[indices]
clean_labels_subset = clean_labels[indices]

print(f"\nUsing {n_samples} samples for TDA")
print(f"Subset shape: {clean_features_subset.shape}")


Using 10000 samples for TDA
Subset shape: (10000, 50)


### Compute Persistent Homology

We'll compute:
- **H0:** Connected components (clusters)
- **H1:** 1-dimensional loops (cycles in the data)

This will 1-2 mins:

In [4]:
result_clean = compute_persistence(
    clean_features_subset,
    maxdim=1,
    verbose=True
)

print("\nPersistence computation successfully completed")

Computing persistence for 10000 points in 50D:
Persistence computed
H0 features: 10000
H1 features: 14978

Persistence computation successfully completed


### Extract Persistence Diagrams

The result contains two diagrams:
- `result_clean['dgms'][0]` → H0 diagram (connected components)
- `result_clean['dgms'][1]` → H1 diagram (loops)

Each diagram is an array of (birth, death) pairs. Let's extract the diagrams:

In [5]:
clean_h0 = result_clean['dgms'][0]
clean_h1 = result_clean['dgms'][1]

print(f"H0 diagram: {len(clean_h0)} features")
print(f"H1 diagram: {len(clean_h1)} features")

print("\n--- H0 Statistics (Connected Components) ---")
h0_stats = get_persistence_stats(clean_h0)
print(f"Number of features: {h0_stats['n_features']}")
print(f"Total persistence: {h0_stats['total_persistence']:.4f}")
print(f"Mean persistence: {h0_stats['mean_persistence']:.4f}")
print(f"Max persistence: {h0_stats['max_persistence']:.4f}")

print("\n--- H1 Statistics (Loops) ---")
h1_stats = get_persistence_stats(clean_h1)
print(f"Number of features: {h1_stats['n_features']}")
print(f"Total persistence: {h1_stats['total_persistence']:.4f}")
print(f"Mean persistence: {h1_stats['mean_persistence']:.4f}")
print(f"Max persistence: {h1_stats['max_persistence']:.4f}")

H0 diagram: 10000 features
H1 diagram: 14978 features

--- H0 Statistics (Connected Components) ---
Number of features: 9999
Total persistence: 2831.9765
Mean persistence: 0.2832
Max persistence: 1.1873

--- H1 Statistics (Loops) ---
Number of features: 14978
Total persistence: 161.5036
Mean persistence: 0.0108
Max persistence: 0.0976
